# Hands-On Assignment 5

In this assignment, you will practice everything that you have learned so far in an end-to-end setting.
You will be provided with a dataset that is **unique to you**, and your task is to perform
all the steps from previous assignments to clean, explore, visualize, and analyze your dataset.

**Written Portion**: Additionally, you will create a report that describes your process and provides insights about your dataset.
Each section that should appear in your report is noted with an orange star (like normal HO tasks).  The report should be  4-6 pages (12 pt font, 1.5 line spacing), and turned in on Canvas as a PDF.

The coding aspect for this assignment will be turned in the same was as all other HO's,
by submitting this file to the autograder.


For this assignment, feel free to make additional functions instead of implementing everything in the provided function.

The objective of this assignment is for you to apply and solidify the skills you have learned in previous assignments.

# Prompt

You have graduated from this class, and are a huge success!
You landed a job doing data science at some fancy company.

You just got a new client with some really interesting problems you get to solve.
Unfortunately, because of a big mess-up on their side the data's metadata got corrupted
(and the person that used to maintain the data just took a vow of silence and moved to a bog).

The only column you are sure about is the `label` column,
which contains a numeric label for each row.
Aside from that, the client does not know anything about the names, content, or even data types for each column.

Your task is to explore, clean, and analyze this data.
You should have already received an email with the details on obtaining your unique data.
Place it in the same directory as this notebook (and your `local_grader.py` script) and name it `data.txt`.

*I know this prompt may sound unrealistic, but I have literally been in a situation exactly like this.
I was working at a database startup, and one of our clients gave us data with over 70 columns and more than a million records and told us:
"The person who used to manage the data is no longer working with us, but this was the data they used to make all their decisions.
We also lost all the metadata information, like column names."
...
Working in industry is not always glamorous.
-Eriq*

# Part 0: Explore Your Data

Before you start doing things to/with your data, it's always a good idea to load up your data and take a look.

In [3]:
import pandas
import pandas as pd
import numpy
import numpy as np
import matplotlib.pyplot as plt
import re
import sklearn.linear_model
import sklearn.neighbors
import sklearn.tree
from sklearn.model_selection import cross_val_score, KFold

# Modify this to point to your data.
unique_data = pandas.read_csv('data.txt', sep = "\t")
unique_data

,label,col_00,col_01,col_02,col_03,col_04,col_05,col_06,col_07,col_08,col_09,col_10,col_11,col_12,col_13,col_14
0,7,1250 mph,0.5773,aNDREW,Stockton,2852 m/s,1.1362,-0.0713,1440,921,801,ROBOTICS ENGINEERING,355,0.2009,-1294.0,302
1,4,1612 mph,1.9812,Lise,Santa Ana,1353 m/s,1.7074,1.3248,1809,-397,1217,Computer Science,328,1.2802,1649.0,-531
2,3,1076 mph,-0.6842,Chris,Moreno Valley,-75 m/s,-0.3871,0.5469,602,-628,373,Computer Engineering,905,-0.7456,1380.0,1231
3,8,1179 mph,0.0825,Vani,Chula Vista,2065 m/s,-0.1814,-0.1667,1634,774,318,Human Computer Interaction,-774,1.132,349.0,-402
4,7,1165 mph,0.7375,Andrew,Anaheim,-219 m/s,0.4305,1.0863,2112,1337,925,Computational Media,-1039,0.0365,-240.0,-58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,7,1601 mph,0.754,Andrew,STOCKTON,-632 m/s,2.9426,0.1795,1400,-598,1241,Biotechnology,-108,-0.3449,1695.0,1532
1196,3,1499 mph,-0.9423,Chris,modesto,1390 m/s,1.6193,2.424,143,759,1800,Computer Engineering,1046,0.2026,1628.0,-388
1197,8,-1104 mph,1.3077,Vani,Chula Vista,-220 m/s,2.2313,0.7703,1858,122,753,Computer Game Design,239,0.8601,1566.0,-302
1198,2,-25 mph,0.7151,NaN,San Jose,NaN,0.6535,0.4165,968,530,2184,Natural Language Processing,6,-0.1845,918.0,433


Don't forget to checkout the column information.

In [11]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   1200 non-null   int64  
 1   col_00  1191 non-null   object 
 2   col_01  1185 non-null   object 
 3   col_02  1190 non-null   object 
 4   col_03  1194 non-null   object 
 5   col_04  1195 non-null   object 
 6   col_05  1189 non-null   object 
 7   col_06  1185 non-null   object 
 8   col_07  1187 non-null   object 
 9   col_08  1193 non-null   object 
 10  col_09  1188 non-null   object 
 11  col_10  1194 non-null   object 
 12  col_11  1193 non-null   object 
 13  col_12  1192 non-null   object 
 14  col_13  1191 non-null   float64
 15  col_14  1187 non-null   object 
dtypes: float64(1), int64(1), object(14)
memory usage: 150.1+ KB


And any numeric information.

In [12]:
unique_data.describe()

,label,col_13
count,1200.000000,1191.000000
mean,4.486667,627.533165
std,2.861814,991.976915
min,0.000000,-2365.000000
25%,2.000000,-27.500000
50%,4.500000,604.000000
75%,7.000000,1313.500000
max,9.000000,3803.000000


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Introduction</h4>

Briefly describe the dataset you’re given and define the goal of the project and how you approach it.
For example, you can present a basic introduction of your data (shape and proposed data types)
and your goal is to use these features to predict the label of the response variable.
Then you propose a few models that are suitable for this project which will be introduced in the modeling section.

# Part 1: Data Cleaning

As always, we should start with data cleaning.
Take what you learned from HO3 to clean up this messy data to a point where it is ready for machine learning algorithms.

Some things you may want to do:
 - Deal with missing/empty values.
 - Fix numeric columns so that they actually contain numbers.
 - Remove inconsistencies from columns.
 - Assign a data type to each column.

<h4 style="color: darkorange; font-size: x-large";>★ Task 1.A</h4>

Complete the following function that takes in a DataFrame and outputs a clean version of the DataFrame.
You can assume that the frame has all the same structure as your unique dataset.
You can return the same or a new data frame.

In [4]:
def clean_data(frame):

    frame = frame.copy()

    for col, val, in frame.items():
        for i, val in enumerate(val):
            if pd.isnull(val):
                frame.at[i, col] = np.nan
            if isinstance(val, str):
                numbers = re.findall(r'[-+]?\d*\.?\d+', val)
                if numbers:
                    numbers = numbers[0]
                    # Float
                    if '.' in numbers:
                        frame.at[i, col] = float(numbers)
                    # Int
                    else:
                        frame.at[i, col] = int(numbers)
                # if everything is a char or a space
                if all(char.isalpha() or char.isspace() for char in val):
                    frame.at[i, col] = str(val)
                elif val == '?':
                    frame.at[i, col] = np.nan

    for column in frame.columns:
        non_nan_values = frame[column].dropna()
        # If its a number
        if non_nan_values.apply(lambda x: isinstance(x, (int, np.integer))).all():
            # If its a float
            if pd.isnull(frame[column]).any():
                frame[column] = frame[column].astype(float)
            # Its a int
            else:
                frame[column] = frame[column].astype(int)
        # Float
        elif non_nan_values.apply(lambda x: isinstance(x, (float, np.floating))).all():
            frame[column] = frame[column].astype(float)
        # String
        else:
            frame[column] = frame[column].astype(str)

    return frame
        

unique_data = clean_data(unique_data)
unique_data

,label,col_00,col_01,col_02,col_03,col_04,col_05,col_06,col_07,col_08,col_09,col_10,col_11,col_12,col_13,col_14
0,7,1250.0,0.5773,aNDREW,Stockton,2852.0,1.1362,-0.0713,1440.0,921.0,801.0,ROBOTICS ENGINEERING,355.0,0.2009,-1294.0,302.0
1,4,1612.0,1.9812,Lise,Santa Ana,1353.0,1.7074,1.3248,1809.0,-397.0,1217.0,Computer Science,328.0,1.2802,1649.0,-531.0
2,3,1076.0,-0.6842,Chris,Moreno Valley,-75.0,-0.3871,0.5469,602.0,-628.0,373.0,Computer Engineering,905.0,-0.7456,1380.0,1231.0
3,8,1179.0,0.0825,Vani,Chula Vista,2065.0,-0.1814,-0.1667,1634.0,774.0,318.0,Human Computer Interaction,-774.0,1.1320,349.0,-402.0
4,7,1165.0,0.7375,Andrew,Anaheim,-219.0,0.4305,1.0863,2112.0,1337.0,925.0,Computational Media,-1039.0,0.0365,-240.0,-58.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,7,1601.0,0.7540,Andrew,STOCKTON,-632.0,2.9426,0.1795,1400.0,-598.0,1241.0,Biotechnology,-108.0,-0.3449,1695.0,1532.0
1196,3,1499.0,-0.9423,Chris,modesto,1390.0,1.6193,2.4240,143.0,759.0,1800.0,Computer Engineering,1046.0,0.2026,1628.0,-388.0
1197,8,-1104.0,1.3077,Vani,Chula Vista,-220.0,2.2313,0.7703,1858.0,122.0,753.0,Computer Game Design,239.0,0.8601,1566.0,-302.0
1198,2,-25.0,0.7151,nan,San Jose,NaN,0.6535,0.4165,968.0,530.0,2184.0,Natural Language Processing,6.0,-0.1845,918.0,433.0


Now we should also be able to view all the numeric columns.

In [14]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   1200 non-null   int64  
 1   col_00  1187 non-null   float64
 2   col_01  1183 non-null   float64
 3   col_02  1200 non-null   object 
 4   col_03  1200 non-null   object 
 5   col_04  1192 non-null   float64
 6   col_05  1188 non-null   float64
 7   col_06  1181 non-null   float64
 8   col_07  1185 non-null   float64
 9   col_08  1192 non-null   float64
 10  col_09  1187 non-null   float64
 11  col_10  1200 non-null   object 
 12  col_11  1192 non-null   float64
 13  col_12  1191 non-null   float64
 14  col_13  1191 non-null   float64
 15  col_14  1183 non-null   float64
dtypes: float64(12), int64(1), object(3)
memory usage: 150.1+ KB


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Cleaning</h4>

Describe the steps you took for data cleaning.
Why did you do this?
Did you have to make some choices along the way? If so, describe them.

# Part 2: Data Visualization

Once you have cleaned up the data, it is time to explore it and find interesting things.
Part of this exploration, will be visualizing the data in a way that makes it easier for yourself and others to understand.
Use what you have learned in HO1 and HO2 to create some visualizations for your dataset.

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Visualization</h4>

Create at least two different visualizations that help describe what you see in your dataset.
Include these visualizations in your report along with descriptions of
how you created the visualization,
what data preparation you had to do for the visualization (aside from the data cleaning in the previous part),
and what the visualization tells us about the data.

In [18]:
majors = ['major_ROBOTICS ENGINEERING', 'major_Computer Science', 'major_Computer Engineering',
                 'major_Human Computer Interaction', 'major_Computational Media',
                 'major_Biotechnology', 'major_Computer Game Design', 'major_Natural Language Processing']

counts = [unique_data[major].sum() for major in majors]

plt.figure(figsize = (8, 8))
plt.pie(counts, labels = majors, autopct = '%1.1f%%', startangle = 140, colors = plt.cm.tab10.colors)
plt.title('Distribution of People in Each Major')
plt.axis('equal')
plt.tight_layout()
plt.show()

KeyError: 'major_ROBOTICS ENGINEERING'

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Visualization</h4>

Create at least two different visualizations that help describe what you see in your dataset.
Include these visualizations in your report along with descriptions of
how you created the visualization,
what data preparation you had to do for the visualization (aside from the data cleaning in the previous part),
and what the visualization tells us about the data.

# Part 3: Modeling

Now that you have a good grasp of your clean data,
it is time to do some machine learning!
(Technically all our previous steps were also machine learning,
but now we get to use classifiers!)

Use the skills you developed to select **three** classifiers and implement them on your data.
For example, you can narrow down your choices to three classifiers which may include:
- Logistic regression
- K-nearest neighbors
- Decision tree
- Or others

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.A</h4>

Complete the following function that takes in no parameters,
and returns a list with **three** untrained classifiers you are going to explore in this assignment.
This method may set parameters/options for the classifiers, but should not do any training/fitting.

For example, if you wanted to use logistic regression,
then **one** of your list items may be:
```
sklearn.linear_model.LogisticRegression()
```

In [5]:
def create_classifiers():
    classifier = []
    classifier.append(sklearn.linear_model.LogisticRegression())
    classifier.append(sklearn.neighbors.KNeighborsClassifier())
    classifier.append(sklearn.tree.DecisionTreeClassifier())
    return classifier

my_classifiers = create_classifiers()
my_classifiers

[LogisticRegression(), KNeighborsClassifier(), DecisionTreeClassifier()]

Now that we have some classifiers, we can see how they perform.

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.B</h4>

Complete the following function that takes in an untrained classifier, a DataFrame, and a number of folds.
This function should run k-fold cross validation with the classifier and the data,
and return a list with the accuracy of each run of cross validation.
You can assume that the frame has the column `label` and the rest of the columns can be considered clean numeric features.

Note that you may have to break your frame into features and labels to do this.
Do not change the passed-in frame (make copies instead).

If you are getting any `ConvergenceWarning`s you may either ignore them,
or try and address them
(they will not affect your autograder score, but may be something to discuss in the written portion of this assignment).

In [6]:
def cross_fold_validation(classifier, frame, folds):
    df_copy = frame.copy()
    df_copy = df_copy.dropna()
    df_encoded = pd.get_dummies(df_copy)
    X = df_encoded.drop('label', axis=1)
    y = df_encoded['label']
    kf = KFold(n_splits=folds)
    accuracy_scores = cross_val_score(classifier, X, y, cv=kf, scoring='accuracy').tolist()
    return accuracy_scores

my_classifiers_scores = []
for classifier in my_classifiers:
    accuracy_scores = cross_fold_validation(classifier, unique_data, 5)
    my_classifiers_scores.append(accuracy_scores)
    print("Classifier: %s, Accuracy: %s." % (type(classifier).__name__, accuracy_scores))

/Users/chucheng_xie/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/chucheng_xie/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linea

Classifier: LogisticRegression, Accuracy: [0.9386792452830188, 0.9528301886792453, 0.9292452830188679, 0.9383886255924171, 0.933649289099526].
Classifier: KNeighborsClassifier, Accuracy: [0.1320754716981132, 0.16037735849056603, 0.19339622641509435, 0.15165876777251186, 0.14218009478672985].
Classifier: DecisionTreeClassifier, Accuracy: [0.8679245283018868, 0.8679245283018868, 0.8632075471698113, 0.8436018957345972, 0.8483412322274881].


<h4 style="color: darkorange; font-size: x-large";>★ Task 3.C</h4>

Complete the following function that takes in two equally-sized lists of numbers and a p-value.
This function should compute whether there is a statistical significance between
these two lists of numbers using a [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test)
at the given p-value.
Return `True` if there is a statistical significance, and `False` otherwise.
Hint: If you wish, you may use the `ttest_ind()` [method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) provided in the scipy package. 

In [10]:
def significance_test(a_values, b_values, p_value):
    x1, x2 = np.mean(a_values), np.mean(b_values)
    n1, n2 = len(a_values), len(b_values)
    std1, std2 = np.std(a_values), np.std(b_values)

    t_statistic = (x1 - x2) / np.sqrt((std1**2 / n1) + (std2**2 / n2))

    val = np.abs(t_statistic) > p_value
    return val

for i in range(len(my_classifiers)):
    for j in range(i + 1, len(my_classifiers)):
        significant = significance_test(my_classifiers_scores[i], my_classifiers_scores[j], 0.10)
        print("%s vs %s: %s" % (type(my_classifiers[i]).__name__,
                                type(my_classifiers[j]).__name__, significant))

LogisticRegression vs KNeighborsClassifier: True
LogisticRegression vs DecisionTreeClassifier: True
KNeighborsClassifier vs DecisionTreeClassifier: True


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Modeling</h4>

Describe the classifiers you have chosen.
Be sure to include all details about any parameter settings used for the algorithms.

Compare the performance of your models using k-fold validation.
You may look at accuracy, F1 or other measures.

Then, briefly summarize your results.
Are your results statistically significant?
Is there a clear winner?
What do the standard deviations look like, and what do they tell us about the different models?
Include a table like Table 1.

<center>Table 1: Every table needs a caption.</center>

| Model | Mean Accuracy | Standard Deviation of Accuracy |
|-------|---------------|--------------------------------|
| Logistic Regression | 0.724 | 0.004
| K-Nearest Neighbor | 0.750 | 0.003
| Decision Tree | 0.655 | 0.011

# Part 4: Analysis

Now, take some time to go over your results for each classifier and try to make sense of them.
 - Why do some classifiers work better than others?
 - Would another evaluation metric work better than vanilla accuracy?
 - Is there still a problem in the data that should fixed in data cleaning?
 - Does the statistical significance between the different classifiers make sense?
 - Are there parameters for the classifier that I can tweak to get better performance?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Analysis</h4>

Discuss your observations, the relationship you found, and how you applied concepts from the class to this project.
For example, you may find that some feature has the most impact in predicting your response variable or removing a feature improves the model accuracy.
Or you may observe that your training accuracy is much higher than your test accuracy and you may want to explain what issues may arise.

# Part 5: Conclusion

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Conclusion</h4>

Briefly summarize the important results and conclusions presented in the project.
What are the important points illustrated by your work?
Are there any areas for further investigation or improvement?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: References</h4>

Include a standard bibliography with citations referring to techniques or published papers you used throughout your report (if you used any).

For example:
```
[1] Derpanopoulos, G. (n.d.). Bayesian Model Checking & Comparison.
https://georgederpa.github.io/teaching/modelChecking.html.
```

# Part XC: Extra Credit

So far you have used a synthetic dataset that was created just for you.
But, data science is always more interesting when you are dealing with actual data from the real world.
Therefore, you will have an opportunity for extra credit on this assignment using real-world data.

For extra credit, repeat the **written tasks** of Parts 0 through 4 with an additional dataset that you find yourself.
For the written portion of the extra credit for Part 0, include information about where you got the data and what the data represents.
You may choose any dataset that represents real data (i.e., is **not** synthetic or generated)
and is **not** [pre-packaged in scikit-learn](https://scikit-learn.org/stable/datasets.html).

Below are some of the many places you can start looking for datasets:
 - [Kaggle](https://www.kaggle.com/datasets) -- Kaggle is a website focused around machine learning competitions,
       where people compete to see who can get the best results on a dataset.
       It is very popular in the machine learning community and has thousands of datasets with descriptions.
       Make sure to read the dataset's description, as Kaggle also has synthetic datasets.
 - [data.gov](https://data.gov/) -- A portal for data from the US government.
        The US government has a lot of data, and much of it has to be available to the public by law.
        This portal contains some of the more organized data from several different government agencies.
        In general, the government has A LOT of interesting data.
        It may not always be clean (remember the CIA factbook), but it is interesting and available.
        All data here should be real-world, but make sure to read the description to verify.
 - [UCI's Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php) -- UC Irvine has their own data repository with a few hundred datasets on many different topics.
        Make sure to read the dataset's description, as UCI also has synthetic datasets.
 - [WHO's Global Health Observatory](https://apps.who.int/gho/data/node.home) -- The World Health Organization keeps track of many different health-related statistics for most of the countries in the world.
        All data here should be real-world, but make sure to read the description to verify.
 - [Google's Dataset Search](https://datasetsearch.research.google.com/) -- Google indexes many datasets that can be searched here.

You can even create a dataset from scratch if you find some data you like that is not already organized into a specific dataset.
The only real distinction between "data" and a "dataset" is that a dataset is organized and finite (has a fixed size).

Create a new section in your written report for this extra credit and include all the written tasks for the extra credit there.
Each written task/section that you complete for your new dataset is eligible for extra credit (so you can still receive some extra credit even if you do not complete all parts).
There is no need to submit any code for the extra credit.
If you created a new dataset, include the dataset or links to it with your submission.